# Combustion Model and Ignition Delay Demo

Written by A. Mark Payne and Alon Grinberg Dana for presentation at ICCK 2017

## User input:

In [ ]:
fuel = 'OC'
equivalence_ratio = 1.0

temperature = 1500.0  # (K)
pressure = 1.0        # (atm)
sim_time = 2          # (ms)
top_sens = 10         # number of top sensitive reactions and thermo to display

rmgpy_path = '../rmg.py' # Change to your rmg.py path

from IPython.display import display, Image
from rmgpy.molecule import Molecule

fuel_molecule = Molecule(smiles=fuel)
print("The fuel molecule is:")
display(fuel_molecule)

## RMG's input file:

In [ ]:
fuel_molecule = Molecule(smiles=fuel)
nC = int(fuel_molecule.get_num_atoms('C'))
nH = int(fuel_molecule.get_num_atoms('H'))
nO = int(fuel_molecule.get_num_atoms('O'))

fuel_stoich = equivalence_ratio/(nC+(nH/4.0)-(nO/2.0))

input_file = f'''
# Data sources
database(
    thermoLibraries = ['BurkeH2O2','primaryThermoLibrary','thermo_DFT_CCSDTF12_BAC','DFT_QCI_thermo','FFCM1(-)'],
    reactionLibraries = ['BurkeH2O2inN2','FFCM1(-)'],
    seedMechanisms = [],
    kineticsDepositories = ['training'],
    kineticsFamilies = 'default',
    kineticsEstimator = 'rate rules',
)

# List of species
species(
    label='fuel',
    reactive=True,
    structure=SMILES('{fuel}'),
)

species(
    label='O2',
    reactive=True,
    structure=SMILES('[O][O]'),
)

species(
    label='N2',
    reactive=False,
    structure=SMILES('N#N'),
)

species(
    label='OH',
    reactive=True,
    structure=SMILES('[OH]'),
)

# Reaction system
simpleReactor(
    temperature=({temperature!s},'K'),
    pressure=({pressure!s},'atm'),
    initialMoleFractions={{
        'fuel': {fuel_stoich!s},
        'O2': 1,
        'N2': 3.76,
    }},
    terminationTime=({sim_time/1000.0},'s'),
    sensitivity=['OH'],
    sensitivityThreshold=0.01,
)

simulator(
    atol=1e-16,
    rtol=1e-8,
    sens_atol=1e-6,
    sens_rtol=1e-4,
)

model(
    toleranceKeepInEdge=0,
    toleranceMoveToCore=0.1,
    toleranceInterruptSimulation=0.1,
    maximumEdgeSpecies=100000,
    filterReactions=True,
    maxNumObjsPerIter=2,
    terminateAtMaxObjects=True,
    maxNumSpecies=50,
)

#pressureDependence(
#        method='modified strong collision',
#        maximumGrainSize=(0.5,'kcal/mol'),
#        minimumNumberOfGrains=250,
#        temperatures=(298,2500,'K',10),
#        pressures=(0.5,3,'bar',5),
#        interpolation=('Chebyshev', 6, 4),
#        maximumAtoms=16,
#)

options(
    units='si',
    generateOutputHTML=True,
    generatePlots=False,
    saveEdgeSpecies=False,
    saveSimulationProfiles=True,
)

generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=5,
    maximumOxygenAtoms=2,
    maximumNitrogenAtoms=0,
    maximumSiliconAtoms=0,
    maximumSulfurAtoms=0,
    maximumHeavyAtoms=6,
    maximumRadicalElectrons=2,
    allowSingletO2=False,
)


'''

import os
import shutil
directory = './rmg_demo'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
input_path = os.path.join(directory, 'input.py')
with open(input_path,'w') as f:
    f.write(input_file)
print('Created RMG input file at ' + os.path.abspath(input_path))

## Run RMG:

In [ ]:
import time
import datetime
import subprocess
start = time.time()

# Execute RMG job
subprocess.check_call(['python', rmgpy_path, input_path])

end = time.time()
print('Total simulation time: ' + str(datetime.timedelta(seconds=round(end-start))))

with open(os.path.join(directory, 'RMG.log'),'r') as f:
    begin = False
    for line in f:
        if 'MODEL GENERATION COMPLETED' in line:
            begin = True
        if begin:
            print(line.strip())

## Run the generated model (using RMG's Cantera functions):

In [ ]:
from rmgpy.chemkin import load_chemkin_file
from rmgpy.tools.canteraModel import Cantera, get_rmg_species_from_user_species
from rmgpy.species import Species
import time

chem_path = os.path.join(directory, 'chemkin')
species_list, reaction_list = load_chemkin_file(os.path.join(chem_path, 'chem_annotated.inp'),
                                              os.path.join(chem_path, 'species_dictionary.txt'),
                                              os.path.join(chem_path, 'tran.dat'))

fuel_species = Species(smiles=fuel)
O2_species = Species(smiles='[O][O]')
N2_species = Species(smiles='N#N')
OH_species = Species(smiles='[OH]')
species_dict = get_rmg_species_from_user_species([fuel_species, O2_species, N2_species, OH_species], species_list)

reactor_type_list = ['IdealGasReactor']
reaction_time_list = ([sim_time], 'ms')

mol_frac_list=[{species_dict[fuel_species]: fuel_stoich,
                species_dict[O2_species]: 1,
                species_dict[N2_species]: 3.76}]
T_list = ([temperature],'K')
P_list = ([pressure],'atm')

job = Cantera(species_list=species_list, reaction_list=reaction_list, output_directory=directory)
job.load_chemkin_model(os.path.join(chem_path, 'chem_annotated.inp'), os.path.join(chem_path, 'tran.dat'))
job.generate_conditions(reactor_type_list, reaction_time_list, mol_frac_list, T_list, P_list)

alldata = job.simulate()
print("Simulation Completed")

## Plot:

In [ ]:
############### Settings ###############
fsize = (8,4) # Change to make the figure fit on your screen
########################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rmgpy.tools import plot as rmg_plot
from operator import itemgetter
%matplotlib notebook

times = alldata[0][0].data
temperatures = alldata[0][1][0].data
pressures = alldata[0][1][1].data

dpdt = (pressures[1:] - pressures[:-1]) / (times[1:] - times[:-1])
idi = next(i for i,d in enumerate(dpdt) if d==max(dpdt))
ign_delay_time = times[idi]

for spc in range(len(alldata[0][1][:])):
    if alldata[0][1][spc].label == str(species_dict[fuel_species]):
        Fuel_idx = spc
    if alldata[0][1][spc].label == str(species_dict[OH_species]):
        OH_idx = spc

for i in range(len(alldata[0][1][Fuel_idx].data)):
    if alldata[0][1][Fuel_idx].data[i]<0.001:
        Fuel_Depletion_Time = times[i]
        break

files = os.listdir(os.path.join(directory, 'solver'))

sensitivity_file = [f for f in files if ('sensitivity' in f) and ('.csv' in f)][0]
SA_time, SA_data = rmg_plot.parse_csv_data(os.path.join(directory, 'solver', sensitivity_file))

time_error = 1

for i in range(len(SA_time.data)):
    if abs(SA_time.data[i]-ign_delay_time)<time_error:
        ign_delay_idx = i
        time_error = abs(SA_time.data[i]-ign_delay_time)

Gidx = 0
for i in range(len(SA_data[:])):
    if "G" in SA_data[i].label:
        if not Gidx:
            Gidx = i
        SA_data[i].label = SA_data[i].label.split('G')[1][1:-1]
    else:
        SA_data[i].label = SA_data[i].label.split(' ')[1]

rank1 = []
for n in range(Gidx):
    rank1.append(abs(SA_data[n].data[ign_delay_idx]))               # list of max SA range for each rxn
num1 = np.linspace(0,len(rank1)-1,len(rank1))        # will be used to get the order of reactions by rank1
num1 = zip(rank1,num1)
num1 = sorted(num1, key=itemgetter(0),reverse=True)
SA_k_data = []
SA_k_label = []
for i in range(min(top_sens, Gidx)):
    SA_k_data.append(SA_data[int(num1[i][1])].data[ign_delay_idx])     # make sorted lists size topSA of SA values and rxns labels
    SA_k_label.append(SA_data[int(num1[i][1])].label)
        
rank2 = []
for n in range(len(SA_data)-Gidx):
    rank2.append(abs(SA_data[n+Gidx].data[ign_delay_idx]))     # list of max SA range for each rxn
num2 = np.linspace(0,len(rank2)-1,len(rank2))        # will be used to get the order of reactions by rank1
num2 = zip(rank2,num2)
num2 = sorted(num2, key=itemgetter(0),reverse=True)
SA_G_data = []
SA_G_label = []
for i in range(min(top_sens, len(SA_data)-Gidx)):
    SA_G_data.append(SA_data[int(num2[i][1])+Gidx].data[ign_delay_idx])     # make sorted lists size topSA of SA values and rxns labels
    SA_G_label.append(SA_data[int(num2[i][1])+Gidx].label)
    
print("Ignition delay time is {0:.4f} ms".format(ign_delay_time * 1000))
        
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.autolayout'] = True

plt.style.use('ggplot')
plt.style.use('seaborn-pastel')

fig = plt.figure(figsize=fsize)

plt.subplot(1,2,1)
plt.plot(alldata[0][0].data*1000, alldata[0][1][Fuel_idx].data,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('$Y_{Fuel}$')
plt.title('Fuel profile')
plt.xlim([0,2000*ign_delay_time])

max_oh = max(alldata[0][1][OH_idx].data)

plt.subplot(1,2,2)
plt.plot(alldata[0][0].data*1000, alldata[0][1][OH_idx].data,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('$Y_{OH}$')
plt.title('OH profile')
plt.xlim([0,2000*ign_delay_time])
plt.arrow(0, alldata[0][1][OH_idx].data[idi], ign_delay_time*1000, 0, width=max_oh*0.01, head_width=max_oh*0.05, head_length=ign_delay_time*120, length_includes_head=True, color='r', shape='full')
plt.annotate(r'$Ignition Delay: \tau_{ign}$', xy=(0,0), xytext=(0, alldata[0][1][OH_idx].data[idi]+0.0005), fontsize=10);

fig = plt.figure(figsize=fsize)

plt.subplot(1,2,1)
plt.plot(alldata[0][0].data*1000, temperatures,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('Temperature (K)')
plt.title('Temperature')
plt.xlim([0,2000*ign_delay_time])

plt.subplot(1,2,2)
plt.plot(alldata[0][0].data*1000, pressures,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('Pressure (Pa)')
plt.title('Pressure')
plt.xlim([0,2000*ign_delay_time])

fig = plt.figure(figsize=fsize)
plt.barh(np.arange(min(top_sens, Gidx)), SA_k_data, 1/1.5, color="blue")
plt.gca().invert_yaxis()
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{[OH]}}{\partial\:\ln{k}}$');
plt.rcParams.update({'axes.labelsize': 20})
plt.yticks(np.arange(min(top_sens, Gidx)),SA_k_label)
plt.title("[OH] sensitivity to kinetics")

fig = plt.figure(figsize=fsize)
plt.barh(np.arange(min(top_sens, len(SA_data)-Gidx)), SA_G_data, 1/1.5, color="blue")
plt.gca().invert_yaxis()
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{[OH]}}{\partial\:G_i}$ $[mol/kcal]$');
plt.rcParams.update({'axes.labelsize': 20})
plt.yticks(np.arange(min(top_sens, len(SA_data)-Gidx)),SA_G_label)
plt.title("[OH] sensitivity to thermo")

plt.show()

In [ ]:
import cantera as ct
gas = ct.Solution(os.path.join(directory, 'cantera', 'chem.cti'))
comp = str(species_dict[fuel_species])+":"+str(fuel_stoich)+","+str(species_dict[O2_species])+":1,"+str(species_dict[N2_species])+":3.76"
gas.TPX = temperature, pressure, comp
reactor = ct.IdealGasConstPressureReactor(gas)
network = ct.ReactorNet([reactor])
network.advance(ign_delay_time)
ROP_C = ct.ReactionPathDiagram(gas, 'C')

from PIL import Image as PILimg
ROP1 = plt.subplot(1,1,1)
dot_file = os.path.join(directory, 'cantera', 'rxnpathC.dot')
img_file = os.path.join(directory, 'cantera', 'rxnpathC.png')
ROP_C.title = 'Reaction path diagram following C'
ROP_C.threshold = 0.01
ROP_C.label_threshold = 0.01
ROP_C.show_details = True
ROP_C.write_dot(dot_file)                                              # write dot file
os.system('dot {0} -Tpng -o{1} -Gdpi=300'.format(dot_file, img_file))  # write png file
fullpath = os.getcwd() + '/' + img_file
display(Image(fullpath))